In [2]:
!pip install torch torchvision torchaudio soundfile

In [3]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-a_db_jxg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-a_db_jxg
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802823 sha256=bdf4de0fb976e9c0a0057495f6f798210eed6d0b13c5cd62be405ccf399320b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-xyn3ynao/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [4]:
!pip install transformers datasets sentencepiece

In [5]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('/kaggle/input/financial-inclusion-speech-dataset/fisd-akuapim-twi-90p/data.csv', delimiter='\t')

In [7]:
data.head()

,Unnamed: 0,Audio Filepath,Transcription,Translation
0,0,lacuna-audios-train/akuapim-twi/audios/Akuapem...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
1,1,lacuna-audios-train/akuapim-twi/audios/Akuapem...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
2,2,lacuna-audios-train/akuapim-twi/audios/Akuapem...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
3,3,lacuna-audios-train/akuapim-twi/audios/Akuapem...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
4,4,lacuna-audios-train/akuapim-twi/audios/Akuapem...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad


In [8]:
# Update the Audio Filepath column
data['Audio Filepath'] = '/kaggle/input/financial-inclusion-speech-dataset/fisd-akuapim-twi-90p/audios/' + data['Audio Filepath'].str.split('/').str[-1]

In [9]:
# Drop the 'Unnamed: 0' column and the index column if it exists
data_cleaned = data.drop(columns=['Unnamed: 0'], errors='ignore')

In [10]:
# Optionally, reset the index if you want a clean index in the new DataFrame
data_cleaned.reset_index(drop=True, inplace=True)

In [11]:
data_cleaned.head()

,Audio Filepath,Transcription,Translation
0,/kaggle/input/financial-inclusion-speech-datas...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
1,/kaggle/input/financial-inclusion-speech-datas...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
2,/kaggle/input/financial-inclusion-speech-datas...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
3,/kaggle/input/financial-inclusion-speech-datas...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad
4,/kaggle/input/financial-inclusion-speech-datas...,Nnipa yɛ bad (informal) Nnipa nnye (formal),People are bad


In [12]:
data_cleaned.to_csv('cleaned_dataset.csv', index=False)

In [13]:
import torch
import torchaudio
from datasets import load_dataset, DatasetDict
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Step 1: Load the Dataset
dataset = load_dataset('csv', data_files='cleaned_dataset.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Audio Filepath', 'Transcription', 'Translation'],
        num_rows: 25636
    })
})

# Preprocess the Audio Files

In [15]:
def preprocess_audio(audio_filepath, target_sample_rate=16000, backend='soundfile'):
    try:
        audio, sample_rate = torchaudio.load(audio_filepath, backend=backend)
        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
            audio = resampler(audio)
        audio = audio.squeeze().numpy()
        return audio
    except FileNotFoundError:
        print(f"Error: File not found at {audio_filepath}")
        return None
    except Exception as e:
        print(f"Error processing {audio_filepath}: {e}")
        return None


# Step 2: Preprocess Dataset

In [16]:
# Safe preprocessing function
def safe_preprocess(example):
    # Check if critical fields are None, and skip processing if they are
    if example['Audio Filepath'] is None or example['Transcription'] is None:
        return None  # Skip this example by returning None
    # Add your preprocessing logic here (e.g., tokenization, normalization)
    # Example:
    example['Transcription'] = example['Transcription'].lower()  # Lowercasing
    return example  # Return processed example if valid


In [17]:
# Map the safe_preprocess function over the dataset
processed_dataset = dataset.map(safe_preprocess)

# Filter out rows that returned None from safe_preprocess
processed_dataset = processed_dataset.filter(lambda x: x is not None)

# Now you can proceed with further processing

Map:   0%|          | 0/25636 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25636 [00:00<?, ? examples/s]

In [18]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['Audio Filepath', 'Transcription', 'Translation'],
        num_rows: 25636
    })
})

# Step 3: Split dataset into train and validation sets

In [19]:
# Assuming 'processed_dataset' is a DatasetDict, get the specific dataset (e.g., 'train')
dataset = processed_dataset['train']  # or the relevant split

# Perform train-test split on the individual dataset
split_dataset = dataset.train_test_split(test_size=0.2)

# If you need to update the DatasetDict with the split data
processed_dataset['train'] = split_dataset['train']
processed_dataset['test'] = split_dataset['test']

In [20]:
processed_dataset['train']

Dataset({
    features: ['Audio Filepath', 'Transcription', 'Translation'],
    num_rows: 20508
})

In [21]:
processed_dataset['test']

Dataset({
    features: ['Audio Filepath', 'Transcription', 'Translation'],
    num_rows: 5128
})

In [22]:
# Check the number of audio files and transcriptions
audio_count = len(dataset['Audio Filepath'])
transcription_count = len(dataset['Transcription'])

# Output the counts
print(f"Number of audio files: {audio_count}")
print(f"Number of transcriptions: {transcription_count}")

# Check if the counts are equal
are_lengths_equal = audio_count == transcription_count
print(f"Are the lengths equal? {are_lengths_equal}")

Number of audio files: 25636
Number of transcriptions: 25636
Are the lengths equal? True


In [23]:
# Check the number of audio files and transcriptions
audio_count = len(processed_dataset['train']['Audio Filepath'])
transcription_count = len(processed_dataset['train']['Transcription'])

# Output the counts
print(f"Number of audio files: {audio_count}")
print(f"Number of transcriptions: {transcription_count}")

# Check if the counts are equal
are_lengths_equal = audio_count == transcription_count
print(f"Are the lengths equal? {are_lengths_equal}")

Number of audio files: 20508
Number of transcriptions: 20508
Are the lengths equal? True


In [24]:
# Check the number of audio files and transcriptions
audio_count = len(processed_dataset['test']['Audio Filepath'])
transcription_count = len(processed_dataset['test']['Transcription'])

# Output the counts
print(f"Number of audio files: {audio_count}")
print(f"Number of transcriptions: {transcription_count}")

# Check if the counts are equal
are_lengths_equal = audio_count == transcription_count
print(f"Are the lengths equal? {are_lengths_equal}")

Number of audio files: 5128
Number of transcriptions: 5128
Are the lengths equal? True


# Step 4: Initialize Whisper Processor and Model

In [25]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [26]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small",task="translate")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [27]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small",task="translate")

In [28]:
print(processed_dataset["train"][0])

{'Audio Filepath': '/kaggle/input/financial-inclusion-speech-dataset/fisd-akuapim-twi-90p/audios/AkuapemTwiFm20-IMMODeKQ-Tmp010-xRUMe6.ogg', 'Transcription': 'apoobɔ nnye', 'Translation': 'Harassment/bullying is not good'}


In [29]:
from datasets import Audio

processed_dataset = processed_dataset.cast_column("Audio Filepath", Audio(sampling_rate=16000))

In [30]:
print(processed_dataset["train"][0])

{'Audio Filepath': {'path': '/kaggle/input/financial-inclusion-speech-dataset/fisd-akuapim-twi-90p/audios/AkuapemTwiFm20-IMMODeKQ-Tmp010-xRUMe6.ogg', 'array': array([-3.05175781e-05, -3.05175781e-05, -3.05175781e-05, ...,
        3.49121094e-02,  3.27453613e-02,  4.23278809e-02]), 'sampling_rate': 16000}, 'Transcription': 'apoobɔ nnye', 'Translation': 'Harassment/bullying is not good'}


In [31]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['Audio Filepath', 'Transcription', 'Translation'],
        num_rows: 20508
    })
    test: Dataset({
        features: ['Audio Filepath', 'Transcription', 'Translation'],
        num_rows: 5128
    })
})

In [32]:
processed_dataset.remove_columns(['Transcription'])

DatasetDict({
    train: Dataset({
        features: ['Audio Filepath', 'Translation'],
        num_rows: 20508
    })
    test: Dataset({
        features: ['Audio Filepath', 'Translation'],
        num_rows: 5128
    })
})

In [33]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["Audio Filepath"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["Translation"]).input_ids
    return batch

In [34]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['Audio Filepath', 'Transcription', 'Translation'],
        num_rows: 20508
    })
    test: Dataset({
        features: ['Audio Filepath', 'Transcription', 'Translation'],
        num_rows: 5128
    })
})

In [35]:
# processed_dataset =processed_dataset.remove_columns(["Transcription"])

# print(processed_dataset)

In [36]:
# from datasets import DatasetDict

# start_index = 11300
# end_index = 11420

# # Function to filter out the specified indices
# def filter_indices(dataset, start, end):
#     # Create a list of indices to keep
#     indices_to_keep = [i for i in range(len(dataset)) if i < start or i > end]
#     return dataset.select(indices_to_keep)

# # Remove the specified indices from the train dataset
# processed_dataset['train'] = filter_indices(processed_dataset['train'], start_index, end_index)

In [37]:
# # Optionally, print the new size of the train dataset
# print(f"New size of train dataset: {len(processed_dataset['train'])}")

In [38]:
# Assuming `processed_dataset` is your DatasetDict
total_examples = 10000

training_examples = total_examples // 2  # 10254

start_index = 0
end_index = start_index + training_examples - 1  # 30538

# Select the training dataset
processed_dataset['train'] = processed_dataset['train'].select(range(start_index, end_index + 1))

# Print the size of the training dataset
print(f"Size of the training dataset: {len(processed_dataset['train'])}")

Size of the training dataset: 5000


In [39]:
processed_dataset = processed_dataset.map(prepare_dataset, remove_columns=processed_dataset.column_names["train"], num_proc=2)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5128 [00:00<?, ? examples/s]

In [40]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5128
    })
})

In [41]:
# # Filter out None entries
# processed_dataset = processed_dataset.filter(lambda x: x is not None)

In [42]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [43]:
# Configure the model's generation settings
model.generation_config.task = "translate" 
model.generation_config.language = None 

In [44]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [45]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [46]:
!pip install evaluate jiwer

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.8 MB/s eta 0:00:0000:0100:01


In [47]:
import evaluate

metric = evaluate.load("wer")

In [48]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [49]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    remove_unused_columns=False
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [50]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [51]:
processor.save_pretrained(training_args.output_dir)

[]

In [52]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5128
    })
})

In [ ]:
# Check for missing 'input_features' in the training dataset
missing_features = [i for i, example in enumerate(processed_dataset['train']) if 'input_features' not in example]

if missing_features:
    print(f"Missing 'input_features' in the following indices: {missing_features}")
else:
    print("All entries have 'input_features'.")

In [55]:
!pip install dill

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
from transformers import WhisperForConditionalGeneration

# Assuming 'model' is your fine-tuned Whisper model
model.save_pretrained("./whisper_finetuned_AkuapemTwi_model")